In [3]:
import requests
import pandas as pd
import json

# CPIC API (약물-유전자 매핑 데이터 제공)
CPIC_URL = "https://api.cpicpgx.org/v1"
headers = {'Accept': 'application/json'}

def fetch_drug_gene_mapping():
    """CPIC API에서 약물-유전자 매핑 데이터 가져오기"""
    
    # 1. 약물 목록 가져오기
    print("📥 약물 목록 가져오는 중...")
    drugs_resp = requests.get(f"{CPIC_URL}/drug", headers=headers)
    if drugs_resp.status_code != 200:
        print(f"❌ 약물 목록 가져오기 실패: {drugs_resp.status_code}")
        return pd.DataFrame()
    
    drugs = drugs_resp.json()
    print(f"  → {len(drugs)}개 약물 로드됨")
    
    # drugid -> name 매핑 생성
    drug_id_to_name = {d['drugid']: d['name'] for d in drugs}
    
    # 2. 약물-유전자 쌍(pair) 가져오기
    print("📥 약물-유전자 매핑 가져오는 중...")
    pairs_resp = requests.get(f"{CPIC_URL}/pair", headers=headers)
    if pairs_resp.status_code != 200:
        print(f"❌ 매핑 가져오기 실패: {pairs_resp.status_code}")
        return pd.DataFrame()
    
    pairs = pairs_resp.json()
    print(f"  → {len(pairs)}개 약물-유전자 쌍 로드됨")
    
    # 3. DataFrame 생성
    records = []
    for pair in pairs:
        drug_id = pair.get('drugid')
        gene = pair.get('genesymbol')
        drug_name = drug_id_to_name.get(drug_id, drug_id)  # ID로 이름 조회
        
        if drug_name and gene:
            records.append({
                'drug': drug_name,
                'gene': gene,
                'drugid': drug_id,
                'cpic_level': pair.get('cpiclevel'),
                'guideline_id': pair.get('guidelineid')
            })
    
    df = pd.DataFrame(records)
    print(f"\n✅ 총 {len(df)}개 약물-유전자 매핑 생성됨")
    return df

# 실행
if __name__ == "__main__":
    df_mapping = fetch_drug_gene_mapping()
    
    if not df_mapping.empty:
        # 결과 미리보기
        print("\n📊 결과 미리보기:")
        print(df_mapping.head(10))
        
        # CSV 저장
        df_mapping.to_csv('cpic_drug_gene_map.csv', index=False)
        print("\n💾 CSV 저장: cpic_drug_gene_map.csv")
        
        # JSON 저장 (약물별 유전자 그룹화)
        grouped = df_mapping.groupby('drug')['gene'].apply(list).to_dict()
        with open('cpic_drug_gene_map.json', 'w', encoding='utf-8') as f:
            json.dump(grouped, f, indent=2, ensure_ascii=False)
        print("💾 JSON 저장: cpic_drug_gene_map.json")
        
        print(f"\n🎉 완료! 총 {df_mapping['drug'].nunique()}개 약물, {df_mapping['gene'].nunique()}개 유전자")


📥 약물 목록 가져오는 중...
  → 323개 약물 로드됨
📥 약물-유전자 매핑 가져오는 중...
  → 634개 약물-유전자 쌍 로드됨

✅ 총 634개 약물-유전자 매핑 생성됨

📊 결과 미리보기:
                                      drug     gene          drugid  \
0                             clomipramine  CYP2C19     RxNorm:2597   
1                         deutetrabenazine   CYP2D6  RxNorm:1876905   
2                               tamsulosin   CYP2D6    RxNorm:77492   
3                      aminosalicylic acid     G6PD     RxNorm:7833   
4                               citalopram   SLC6A4     RxNorm:2556   
5                             esomeprazole  CYP2C19   RxNorm:283742   
6                                ibuprofen   CYP2C8     RxNorm:5640   
7                                methadone   CYP2D6     RxNorm:6813   
8                              norfloxacin     G6PD     RxNorm:7517   
9  Selective serotonin reuptake inhibitors     COMT       ATC:N06AB   

  cpic_level  guideline_id  
0          B      100414.0  
1        B/C           NaN  
2        B/C     